In [1]:
from datasets import load_from_disk
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback, AutoTokenizer
from huggingface_hub import login
from dotenv import load_dotenv
import os
import wandb

In [2]:
WB_KEY = os.getenv("WB_KEY")
wandb.login(key=WB_KEY)
run = wandb.init(project="Digital Self-Replica", job_type="Training", name="train_test2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Francesco\_netrc
wandb: Currently logged in as: francescobrigante (francescobrigante_s_projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
#pip install -U bitsandbytes

In [4]:
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
if (HF_TOKEN == None):
    raise ValueError("HF_TOKEN is not set")
login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
# 4 bit quantization
# could be further increased to 8b for more precision
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# LoRA configuration for Qwen model architecture
lora_config = LoraConfig(
    r=32,                       #rank of the added low-rank matrices
    lora_alpha=64,              #generally 2*r
    target_modules=[            #modules where LoRA is applied
        "q_proj",               # query, key, value, output projection layers in the self-attention
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",            # gate, up, down are part of the FFNN in the model
        "up_proj",
        "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [6]:
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# # preparing model for LoRA
# model = prepare_model_for_kbit_training(model)
# model = get_peft_model(model, lora_config)


# # training arguments
# training_args = TrainingArguments(
#     output_dir="./francesco_lora",
#     num_train_epochs=3,
#     per_device_train_batch_size=6,
#     gradient_accumulation_steps=4,      # effective batch size = per_device_train_batch_size * gradinet_accumulation_steps
#     #per_device_eval_batch_size=4,
#     #eval_accumulation_steps=6,
#     # warmup_steps=5,
#     warmup_ratio=0.03,
#     learning_rate=3e-4,                # Slightly lower for distilled model
#     optim="paged_adamw_8bit",         # 8bit optimizer <- ADDED
#     #optim="adamw_torch",
#     lr_scheduler_type="cosine",       # cosine learning rate scheduler <- ADDED
#     fp16=True,
#     logging_steps=5,
#     eval_strategy="no",
#     #eval_strategy="steps",
#     #eval_steps=5,
#     save_steps=50,
#     save_strategy="steps",
#     #load_best_model_at_end=True,
#     #save_total_limit=1,
#     metric_for_best_model="loss",
#     greater_is_better=False,            #lower loss is better
#     gradient_checkpointing=False,
#     disable_tqdm=False,
#     report_to=["wandb"],                                # Enable W&B logging 
#     label_names=["labels"]  # Explicitly specify label field
# )

In [8]:
# preparing model for LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)


# TRY  AND BATCH 8x8

# TRY ALSO THIS:

    # eval_strategy="steps",
    # eval_steps=100,
    # per_device_eval_batch_size=16,
    # eval_accumulation_steps=2,
    # compute_metrics=None,  # Loss-only evaluation
    # load_best_model_at_end=True,

# training arguments
training_args = TrainingArguments(
    output_dir="./francesco_lora",
    num_train_epochs=3,
    per_device_train_batch_size=6,
    gradient_accumulation_steps=4,      # effective batch size = per_device_train_batch_size * gradinet_accumulation_steps
    #per_device_eval_batch_size=4,
    #eval_accumulation_steps=6,
    # warmup_steps=5,
    warmup_ratio=0.1,
    learning_rate=2e-4,                # Slightly lower for distilled model
    optim="paged_adamw_8bit",         # 8bit optimizer <- ADDED
    #optim="adamw_torch",
    lr_scheduler_type="cosine",       # cosine learning rate scheduler <- ADDED
    weight_decay=0.01,
    fp16=True,
    logging_steps=5,
    #eval_strategy="no",
    eval_strategy="steps",
    per_device_eval_batch_size=16,
    eval_accumulation_steps=2,
    eval_steps=5,
    save_steps=50,
    save_strategy="steps",
    load_best_model_at_end=True,
    #save_total_limit=1,
    metric_for_best_model="loss",
    greater_is_better=False,            #lower loss is better
    gradient_checkpointing=False,
    max_grad_norm=0.5,
    disable_tqdm=False,
    report_to=["wandb"],                                # Enable W&B logging
    label_names=["labels"]  # Explicitly specify label field
)

In [9]:
from transformers import DataCollatorWithPadding, DataCollatorForLanguageModeling, DefaultDataCollator

tokenizer = AutoTokenizer.from_pretrained(model_id)

data_collator = DefaultDataCollator()

#Dynamic padding for causal LM
# data_collator = DataCollatorForLanguageModeling(
#    tokenizer=tokenizer,
#    mlm=False
# )

#Dynamic padding more general purpose
# it doesnt handle shifting lables: you have to implement it manually

# data_collator = DataCollatorWithPadding(
#     tokenizer=tokenizer,
#     padding=True,            # pad to longest in batch
#     return_tensors="pt",
# )


In [10]:
# loading datasets
tokenized_train = load_from_disk('datasets/tokenized_train')
tokenized_val = load_from_disk('datasets/tokenized_val')
#tokenized_test = load_from_disk('datasets/tokenized_test')

# print
print(f"Training examples: {len(tokenized_train)}")
print(f"Validation examples: {len(tokenized_val)}")
#print(f"Test examples: {len(tokenized_test)}")

print("\nOne training example:")
print(tokenized_train[1000])

Training examples: 8720
Validation examples: 1090

One training example:
{'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 

In [11]:
#IF ON COLAB
# from google.colab import drive

# drive.mount('/content/drive')
# drive_base_path = '/content/drive/My Drive/datasets'

# tokenized_train = load_from_disk(os.path.join(drive_base_path, 'tokenized_train'))
# tokenized_val = load_from_disk(os.path.join(drive_base_path, 'tokenized_val'))
# tokenized_test = load_from_disk(os.path.join(drive_base_path, 'tokenized_test'))

# print("Datasets loaded successfully from Google Drive!")
# print(f"Training examples: {len(tokenized_train)}")
# print(f"Validation examples: {len(tokenized_val)}")
# print(f"Test examples: {len(tokenized_test)}")

# print("\nOne training example:")
# print(tokenized_train[8000])

In [12]:
# print trainable parameters
model.print_trainable_parameters()

# training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

# add early stopping
early_stopping = EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)
trainer.add_callback(early_stopping)

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


  0%|          | 0/1089 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\c

{'loss': 8.532, 'grad_norm': 13.746661186218262, 'learning_rate': 9.174311926605506e-06, 'epoch': 0.01}


  0%|          | 0/69 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
#trainer.train(resume_from_checkpoint="./francesco_lora/checkpoint-50")

	save_steps: 50 (from args) != 25 (from trainer_state.json)


  0%|          | 0/1089 [00:00<?, ?it/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\c

{'loss': 2.9748, 'grad_norm': 2.294355630874634, 'learning_rate': 4.545454545454545e-05, 'epoch': 0.08}
{'loss': 2.6041, 'grad_norm': 2.8884832859039307, 'learning_rate': 9.09090909090909e-05, 'epoch': 0.1}
{'loss': 2.4398, 'grad_norm': 3.4224636554718018, 'learning_rate': 0.00013636363636363634, 'epoch': 0.11}
{'loss': 2.9799, 'grad_norm': 4.153173923492432, 'learning_rate': 0.0001818181818181818, 'epoch': 0.12}
{'loss': 3.3979, 'grad_norm': 5.842740058898926, 'learning_rate': 0.00022727272727272725, 'epoch': 0.14}


c:\Users\Francesco\anaconda3\lib\site-packages\peft\utils\other.py:689: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: c3490581-176d-47df-9103-33216b49734d)') - silently ignoring the lookup for the file config.json in deepseek-ai/DeepSeek-R1-Distill-Qwen-7B.
  warnings.warn(
c:\Users\Francesco\anaconda3\lib\site-packages\peft\utils\save_and_load.py:243: UserWarning: Could not find a config file in deepseek-ai/DeepSeek-R1-Distill-Qwen-7B - will assume that the vocabulary was not modified.
  warnings.warn(
c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you

...

c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


{'loss': 2.5841, 'grad_norm': 3.4312779903411865, 'learning_rate': 0.00013042107116699228, 'epoch': 1.73}
{'loss': 2.527, 'grad_norm': 3.25289249420166, 'learning_rate': 0.0001282111625926641, 'epoch': 1.75}
{'loss': 2.4421, 'grad_norm': 2.446511745452881, 'learning_rate': 0.00012600607502762096, 'epoch': 1.76}
{'loss': 2.5528, 'grad_norm': 2.9619667530059814, 'learning_rate': 0.00012380629637066297, 'epoch': 1.77}
{'loss': 2.8119, 'grad_norm': 3.2970638275146484, 'learning_rate': 0.00012161231334593851, 'epoch': 1.79}


c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


{'loss': 2.5309, 'grad_norm': 2.570861339569092, 'learning_rate': 0.00011942461139525123, 'epoch': 1.8}
{'loss': 2.6826, 'grad_norm': 2.899683952331543, 'learning_rate': 0.00011724367457065135, 'epoch': 1.82}
{'loss': 2.5655, 'grad_norm': 2.263598680496216, 'learning_rate': 0.00011506998542733373, 'epoch': 1.83}
{'loss': 2.6892, 'grad_norm': 2.794584274291992, 'learning_rate': 0.00011290402491686766, 'epoch': 1.84}
{'loss': 2.6247, 'grad_norm': 3.17472767829895, 'learning_rate': 0.0001107462722807811, 'epoch': 1.86}


c:\Users\Francesco\anaconda3\lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
c:\Users\Francesco\anaconda3\lib\site-packages\torch\utils\checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
